In [ ]:
from libraries import *
from parameters import *
import anndata as ad

In [ ]:
%load_ext rpy2.ipython

In [ ]:
os.getcwd()
os.chdir(projectDir)

In [ ]:
# adataLPSMinus = sc.read_10x_h5('./data/data_unperturbed/LPSMinus_v3_raw_feature_bc_matrix.h5').copy()
# adataLPSPlus = sc.read_10x_h5('./data/data_unperturbed/LPSPlus_v3_raw_feature_bc_matrix.h5').copy()
# adataLPSMinus.var_names_make_unique()
# adataLPSPlus.var_names_make_unique()


# sc.pp.filter_cells(adataLPSMinus, min_genes=800)
# sc.pp.filter_genes(adataLPSMinus, min_cells=10)
# sc.pp.filter_cells(adataLPSPlus, min_genes=800)
# sc.pp.filter_genes(adataLPSPlus, min_cells=10)


# adataLPSMinus.var['mt'] = adataLPSMinus.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
# sc.pp.calculate_qc_metrics(adataLPSMinus, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
# adataLPSPlus.var['mt'] = adataLPSPlus.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
# sc.pp.calculate_qc_metrics(adataLPSPlus, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)


# sc.pl.violin(adataLPSMinus, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
#              jitter=0.4, multi_panel=True)

# sc.pl.violin(adataLPSPlus, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
#              jitter=0.4, multi_panel=True)


# # sc.pl.scatter(adataLPSPlus, x='total_counts', y='pct_counts_mt')
# # sc.pl.scatter(adataLPSPlus, x='total_counts', y='n_genes_by_counts')
# # sc.pl.scatter(adataLPSMinus, x='total_counts', y='pct_counts_mt')
# # sc.pl.scatter(adataLPSMinus, x='total_counts', y='n_genes_by_counts')


# adataLPSPlus = adataLPSPlus[adataLPSPlus.obs.n_genes_by_counts < 2500, :]
# adataLPSPlus = adataLPSPlus[adataLPSPlus.obs.pct_counts_mt < 10, :]
# adataLPSMinus = adataLPSMinus[adataLPSMinus.obs.n_genes_by_counts < 2500, :]
# adataLPSMinus = adataLPSMinus[adataLPSMinus.obs.pct_counts_mt < 10, :]


# adataLPSPlus.obs["type"] = "LPSPlus"
# adataLPSMinus.obs["type"] = "LPSMinus"


# adataAll = ad.concat([adataLPSMinus, adataLPSPlus], join="outer")
# #adataAll.var_names_make_unique()
# adataAll.obs_names_make_unique()

# sc.pp.normalize_total(adataAll, target_sum=1e4)
# sc.pp.log1p(adataAll)
# adataAll.raw = adataAll


# gene_list_url = 'https://raw.githubusercontent.com/theislab/scanpy_usage/master/180209_cell_cycle/data/regev_lab_cell_cycle_genes.txt'

# cell_cycle_genes = [str(x.strip(), 'utf-8').capitalize() for x in urlopen(gene_list_url)] # capitalize = shame


# s_genes = cell_cycle_genes[:43]
# g2m_genes = cell_cycle_genes[43:]


# sc.tl.score_genes_cell_cycle(adataAll, s_genes=s_genes, g2m_genes=g2m_genes)


# sc.pp.highly_variable_genes(adataAll, n_top_genes = 2000)
# sc.pp.regress_out(adataAll, ['total_counts', 'pct_counts_mt'])
# sc.pp.scale(adataAll, max_value=10)
# sc.tl.pca(adataAll, svd_solver='arpack')
# sc.pp.neighbors(adataAll, n_neighbors=10, n_pcs=50)

# sc.tl.umap(adataAll, min_dist=0.4, init_pos="spectral")

# sc.tl.leiden(adataAll, resolution=0.5)

# sc.tl.diffmap(adataAll)
# sc.tl.rank_genes_groups(adataAll, groupby="leiden", n_genes=2000, method="t-test_overestim_var")


#adataAll.write("outputs/anndata/adataLPSPlusMinus.h5ad")

In [ ]:
adata = sc.read(par_save_filename_8)

In [ ]:
adataAll = sc.read("outputs/anndata/adataLPSPlusMinus.h5ad")

In [ ]:
dcGenes = pd.read_csv('./PositiveControls/DC_cellstate_genes.csv')
dc1Genes = dcGenes["DC1 genes"].unique()
dc2Genes = dcGenes["DC2 genes"].unique()
mregGenes = dcGenes["mregDC genes"].unique()
macGenes = dcGenes["Macrophage genes"].unique()

leidenMarkersOfPerturbedCells = pd.read_csv("./TextFiles/Leiden_top100Markers.csv")
leidenMarkersOfPerturbedCells = leidenMarkersOfPerturbedCells.groupby('group').head(50)



In [ ]:
sc.tl.score_genes(adata=adata, gene_list=dc1Genes, score_name="DC1")
sc.tl.score_genes(adata=adata, gene_list=dc2Genes, score_name="DC2")
sc.tl.score_genes(adata=adata, gene_list=mregGenes, score_name="mreg")
sc.tl.score_genes(adata=adata, gene_list=macGenes, score_name="Mac")


sc.tl.score_genes(adata=adataAll, gene_list=dc1Genes, score_name="DC1")
sc.tl.score_genes(adata=adataAll, gene_list=dc2Genes, score_name="DC2")
sc.tl.score_genes(adata=adataAll, gene_list=mregGenes, score_name="mreg")
sc.tl.score_genes(adata=adataAll, gene_list=macGenes, score_name="Mac")

In [ ]:
adata.obs["subCellType"] = "DC2"
adata.obs.loc[adata.obs.leiden.isin(['3']), "subCellType"] = "MacDC"
adata.obs.loc[adata.obs.leiden.isin(['8']), "subCellType"] = "DC1"
adata.obs.loc[adata.obs.leiden.isin(['5']), "subCellType"] = "MReg"

In [ ]:
for i in leidenMarkersOfPerturbedCells.group.unique():
     myGeneList = leidenMarkersOfPerturbedCells.loc[leidenMarkersOfPerturbedCells.group == i,'names']
     myGeneList = [x.replace(".","-") for x in myGeneList]
     sc.tl.score_genes(adata=adataAll, gene_list=myGeneList, score_name="TC_"+str(i) + "_scores")
     sc.tl.score_genes(adata=adata, gene_list=myGeneList, score_name="TC_"+str(i) + "_scores")


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
X_train = adata.obs[["DC1", "DC2", "mreg", "Mac"]]
y_train = adata.obs["subCellType"]

scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled = scaler.transform(X_train)

clf = LogisticRegression(random_state=0).fit(X_scaled, y_train)

X_test = adataAll.obs[["DC1", "DC2", "mreg", "Mac"]]
scaler_test = preprocessing.StandardScaler().fit(X_test)
X_scaled_test = scaler.transform(X_test)

adataAll.obs["Pred_subCellType"] = clf.predict(X_scaled_test)

In [ ]:
X_train = adata.obs[["TC_0_scores", "TC_1_scores", "TC_2_scores", "TC_3_scores",
                    "TC_4_scores", "TC_5_scores", "TC_6_scores", "TC_7_scores",
                    "TC_8_scores", "TC_9_scores"]]
y_train = adata.obs["leiden"]

scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled = scaler.transform(X_train)

clf = LogisticRegression(random_state=0).fit(X_scaled, y_train)

X_test = adataAll.obs[["TC_0_scores", "TC_1_scores", "TC_2_scores", "TC_3_scores",
                    "TC_4_scores", "TC_5_scores", "TC_6_scores", "TC_7_scores",
                    "TC_8_scores", "TC_9_scores"]]
scaler_test = preprocessing.StandardScaler().fit(X_test)
X_scaled_test = scaler.transform(X_test)

adataAll.obs["Pred_TC"] = clf.predict(X_scaled_test)

In [ ]:
sc.pl.umap(adataAll, color='type', color_map="coolwarm", 
           legend_loc='on data', legend_fontoutline=3, 
           legend_fontsize=14, legend_fontweight='normal', size=3, save="Figure_S1H.pdf");

In [ ]:
sc.pl.umap(adataAll, color='phase', palette = "Paired",save="Figure_S1I.pdf" )

In [ ]:
f, ax = plt.subplots(figsize=(4, 4))
sc.pl.umap(adataAll, color='Pred_TC', 
           legend_fontoutline=3, legend_fontsize=14, 
           legend_fontweight='normal', title='Predicted TCs',
           ax=ax, show=False, size=3, save="Figure_S1U.pdf");

In [ ]:
f, ax = plt.subplots(figsize=(4, 4))
sc.pl.umap(adataAll, color='Pred_subCellType', 
           legend_fontoutline=3, legend_fontsize=14, 
           legend_fontweight='normal', title='Predicted sub-cell types',
           ax=ax, show=False, size=10, palette="Accent", save="Figure_S1W.pdf");

In [ ]:
for i in leidenMarkersOfPerturbedCells.group.unique():
     sc.pl.umap(adataAll, color="TC_"+str(i) + "_scores", size=4, color_map="seismic", save="TC_"+str(i))
     #sc.pl.violin(adataAll, "TC_"+str(i)+"_scores", groupby='leiden')

In [ ]:
kk = adataAll.obs[["type","Pred_TC","Pred_subCellType"]]

In [ ]:
hehe = pd.DataFrame(adata.obs["leiden"].value_counts())
hehe["Pred_TC"] = hehe.index

In [ ]:
%%R -i kk,hehe -w 7 -h 6 -u in

library("data.table")
library(ggplot2) 

hehe$Perc = hehe$leiden/sum(hehe$leiden)
hehe$type="Perturbed"
hehe=hehe[,c("type", "Pred_TC", "Perc")]

predTCs = data.table(table(kk[,c("type", "Pred_TC")]))
predTCs[,Perc:=(N/sum(N)),by=type]
predTCs = data.frame(predTCs)
predTCs[predTCs$type == "LPSMinus","type"] = "LPS-"
predTCs[predTCs$type == "LPSPlus","type"] = "LPS+"
predTCs$N=NULL

predTCs = rbind(predTCs, hehe)

head(predTCs)

#pdf(file="Figure_S1V.pdf", width=40, height=20)

ggplot(data=predTCs, 
       aes(x=type, y=Perc, fill=Pred_TC)) + theme_minimal()+
       facet_wrap(~Pred_TC,ncol=3 )+
       geom_bar(stat="identity")+
       scale_fill_manual(name = "Predicted Transcription Cluster", 
                         values=c('#1f77b4', '#ff7f0e', '#279e68', '#d62728', '#aa40fc', 
                                                       '#8c564b','#e377c2', '#b5bd61', '#17becf', '#aec7e8' ))+
       ylab("Percent of cells") + xlab("") +
      theme(axis.text = element_text(size=13), axis.title = element_text(size=13), 
      legend.text=element_text(size=13), legend.title=element_text(size=11), legend.pos="top")

#dev.off()
#  
# library(reshape2) 
# 
# library(ggpubr) 

# ldMelted = melt(leidenScores, id.vars="type")
# 

# ggplot(ldMelted, aes(x=type, y=value)) + 
#      geom_violin(trim=FALSE, aes(fill=type))+
#      geom_boxplot()+theme_minimal()+ 
#      facet_wrap(.~variable, ncol=5)+
#      #stat_summary(fun.data = mean, geom = "text", size=6)+
#         stat_compare_means(method="wilcox.test",paired = FALSE,
#                                 comparisons = list(c("LPSMinus", "LPSPlus")),
#                                 color="red", method.args = list(alternative = "less"),
#                                 label.y = c(3), aes(label = ..p.signif..))+
#      scale_fill_brewer(palette = "Pastel1")+
# ylab("Cell score")+
# theme(axis.text = element_text(size=15),strip.text=element_text(size=12),
#               axis.title =  element_text(size=16))+ylim(0,5) +xlab("") 

In [ ]:
hehe = pd.DataFrame(adata.obs["subCellType"].value_counts())
hehe["Pred_subCellType"] = hehe.index
hehe

In [ ]:
%%R -i kk,hehe -w 9 -h 4 -u in

predTCs = data.table(table(kk[,c("type", "Pred_subCellType")]))
predTCs[,Perc:=(N/sum(N)),by=type]
predTCs = data.frame(predTCs)
predTCs


In [ ]:
%%R -i kk,hehe -w 9 -h 4 -u in

library("data.table")
library(ggplot2) 

head(hehe)
hehe$Perc = hehe$subCellType/sum(hehe$subCellType)
hehe$type="Perturbed"
hehe=hehe[,c("type", "Pred_subCellType", "Perc")]


predTCs = data.table(table(kk[,c("type", "Pred_subCellType")]))
predTCs[,Perc:=(N/sum(N)),by=type]
predTCs = data.frame(predTCs)
predTCs[predTCs$type == "LPSMinus","type"] = "LPS-"
predTCs[predTCs$type == "LPSPlus","type"] = "LPS+"
predTCs$N = NULL

predTCs = rbind(predTCs, hehe)


options(repr.plot.width=3, repr.plot.height=2)
ggplot(data=predTCs, 
       aes(x=type, y=Perc, fill=Pred_subCellType)) + theme_minimal()+
       facet_wrap(~Pred_subCellType,ncol=4 )+
       geom_bar(stat="identity")+ scale_fill_brewer(palette="Dark2")+
       ylab("Percent of cells") + xlab("") +
      theme(axis.text = element_text(size=13), axis.title = element_text(size=13), 
            strip.text = element_text(size=15), 
      legend.text=element_text(size=13), legend.title=element_text(size=11), legend.pos="top")



In [ ]:
allDCgenes = np.concatenate((dc1Genes, dc2Genes, mregGenes))
sc.tl.score_genes(adata=adataAll, gene_list=allDCgenes, score_name="DCSig")

In [ ]:
sc.pl.umap(adataAll, color="DCSig", size=4, color_map="coolwarm", vmin=-0.2, vmax=0.3)

In [ ]:
adataAll.obs["DCSig_zscore"] = scipy.stats.zscore(adataAll.obs["DCSig"])
adataAll.obs["Mac_zscore"] = scipy.stats.zscore(adataAll.obs["Mac"])

In [ ]:
adataAll.obs["MACoverDC"] = adataAll.obs["Mac_zscore"] - adataAll.obs["DCSig_zscore"]

In [ ]:
sc.pl.umap(adataAll, color="MACoverDC", 
           size=4, 
           color_map="PiYG", 
           vmin=-3, 
           vmax=3)


In [ ]:
leidenScores = adataAll.obs[["TC_"+str(x)+"_scores" for x in range(0,10)]+["type"]]

In [ ]:
%%R -i leidenScores -w 13 -h 6 -u in

library(MASS) 
library(reshape2) 
library(ggplot2) 
library(ggpubr) 

leidenScores$type = as.character(leidenScores$type)

leidenScores[leidenScores$type == "LPSMinus","type"] = "LPS-"
leidenScores[leidenScores$type == "LPSPlus","type"] = "LPS+"
head(leidenScores)

ldMelted = melt(leidenScores, id.vars="type")
options(repr.plot.width=6, repr.plot.height=4)

ggplot(ldMelted, aes(x=type, y=value)) + 
     geom_violin(trim=FALSE, aes(fill=type))+
     geom_boxplot()+theme_minimal()+ 
     facet_wrap(.~variable, ncol=5)+
     #stat_summary(fun.data = mean, geom = "text", size=6)+
        stat_compare_means(method="wilcox.test",paired = FALSE,
                                comparisons = list(c("LPS-", "LPS+")),
                                color="red", method.args = list(alternative = "less"),
                                label.y = c(3), aes(label = ..p.signif..))+
     scale_fill_brewer(palette = "Pastel1")+
ylab("Cell score")+
theme(axis.text = element_text(size=15),strip.text=element_text(size=12),
              axis.title =  element_text(size=16))+ylim(0,5) +xlab("") 

In [ ]:
%%R -i cellScoresOfStates -w 13 -h 6 -u in

cellScorMelted = melt(cellScoresOfStates)
head(cellScorMelted)
#options(repr.plot.width=6, repr.plot.height=4)

ggplot(cellScorMelted, aes(x=variable, y=value)) + 
     geom_violin(trim=FALSE, aes(fill=variable))+
     geom_boxplot()+theme_minimal()
        

#      facet_wrap(.~variable, ncol=5)+
#     stat_summary(fun.data = mean, geom = "text", size=6)
#         stat_compare_means(method="wilcox.test",paired = FALSE,
#                                 comparisons = list(c("LPSMinus", "LPSPlus")),
#                                 color="red", method.args = list(alternative = "less"),
#                                 label.y = c(3), aes(label = ..p.signif..))+
#      scale_fill_brewer(palette = "Pastel1")+
# ylab("Cell score")+
# theme(axis.text = element_text(size=15),strip.text=element_text(size=12),
#               axis.title =  element_text(size=16))+ylim(0,5) +xlab("") 